# E6 Vector memory lets you manipulate information in a new kind of way

## A quick business review (because the technology alone is not enough)

Every business benefits from four courses of action to be used together or in combination:

1. Grow the existing business
2. Save money and time
3. Add completely new business
4. Prepare for the unknown

![](assets/shopkeeper.png)

The business owner is always leaking time and money from their valuable buckets of time and money. Let's help them succeed!

## 🔥 Get a kernel ready with a vector memory store

We're going to create a kernel that includes an embedding generation service — so that we can best construct the context that will be used.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", api_key, endpoint))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-4", api_key, org_id))
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory='mymemories'))
#kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

## 😶‍🌫️ Build up our vector database with information to draw upon later

We store the SWOT information in "conventional" arrays in Python that are not "semantic" in nature. That said, they're still eminently useful for the kind of computations we'll play with.

In [ ]:
from IPython.display import display, Markdown

strength_questions = ["What unique recipes or ingredients does the pizza shop use?","What are the skills and experience of the staff?","Does the pizza shop have a strong reputation in the local area?","Are there any unique features of the shop or its location that attract customers?", "Does the pizza shop have a strong reputation in the local area?", "Are there any unique features of the shop or its location that attract customers?"]
weakness_questions = ["What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)","Are there financial constraints that limit growth or improvements?","Are there any gaps in the product offering?","Are there customer complaints or negative reviews that need to be addressed?"]
opportunities_questions = ["Is there potential for new products or services (e.g., catering, delivery)?","Are there under-served customer segments or market areas?","Can new technologies or systems enhance the business operations?","Are there partnerships or local events that can be leveraged for marketing?"]
threats_questions = ["Who are the major competitors and what are they offering?","Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?","Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?","Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"]

strengths = [ "Unique garlic pizza recipe that wins top awards","Owner trained in Sicily at some of the best pizzerias","Strong local reputation","Prime location on university campus" ]
weaknesses = [ "High staff turnover","Floods in the area damaged the seating areas that are in need of repair","Absence of popular calzones from menu","Negative reviews from younger demographic for lack of hip ingredients" ]
opportunities = [ "Untapped catering potential","Growing local tech startup community","Unexplored online presence and order capabilities","Upcoming annual food fair" ]
threats = [ "Competition from cheaper pizza businesses nearby","There's nearby street construction that will impact foot traffic","Rising cost of cheese will increase the cost of pizzas","No immediate local regulatory changes but it's election season" ]

print("✅ SWOT analysis for the pizza shop is resident in native memory")

memoryCollectionName = "SWOT"

for i in range(len(strengths)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"strength-{i}", text=f"Internal business strength (S in SWOT) that makes customers happy and satisfied Q&A: Q: {strength_questions[i]} A: {strengths[i]}")
for i in range(len(weaknesses)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"weakness-{i}", text=f"Internal business weakness (W in SWOT) that makes customers unhappy and dissatisfied Q&A: Q: {weakness_questions[i]} A: {weaknesses[i]}")
for i in range(len(opportunities)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"opportunity-{i}", text=f"External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: {opportunities_questions[i]} A: {opportunities[i]}")
for i in range(len(threats)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"threat-{i}", text=f"External threat (T in SWOT) to the business that impacts its survival Q&A: Q: {threats_questions[i]} A: {threats[i]}")

print("😶‍🌫️ Embeddings for SWOT have been generated")

skillsDirectory = "./plugins-sk"

pluginBT = kernel.import_semantic_skill_from_directory(skillsDirectory, "BusinessThinking");

my_context = kernel.create_new_context()

the_business = 'makes pizzas'
my_context['input'] = the_business
my_context['strengths'] = ", ".join(strengths)
my_context['weaknesses'] = ", ".join(weaknesses)
my_context['opportunities'] = ", ".join(opportunities)
my_context['threats'] = ", ".join(threats)

print("🤖 Computing cost efficiency...")
costefficiency_result = await kernel.run_async(pluginBT["SeekCostEfficiency"], input_context=my_context)
costefficiency_str = str("## Suggestions for how to gain cost efficiencies\n" + str(costefficiency_result))
await kernel.memory.save_information_async(memoryCollectionName, id="costefficiency", text=costefficiency_str)

print("🤖 Computing time efficiency...")
timeefficiency_result = await kernel.run_async(pluginBT["SeekTimeEfficiency"], input_context=my_context)
timeefficiency_str = "## Suggestions for how to gain time efficiencies\n"+str(timeefficiency_result)
await kernel.memory.save_information_async(memoryCollectionName, id="timeefficiency", text=timeefficiency_str)

print("🤖 Computing business strategies overview...")
bizstrat_result = await kernel.run_async(pluginBT["BasicStrategies"], input_context=my_context)
bizstrat_str = "# Business strategy thinking based on SWOT analysis\n"+str(bizstrat_result)
await kernel.memory.save_information_async(memoryCollectionName, id="bizstrat", text=bizstrat_str)

customer_comments = """
Customer 1: The seats look really raggedy.
Customer 2: The garlic pizza is the best on this earth.
Customer 3: I've noticed that there's a new server every time I visit, and they're clueless.
Customer 4: Why aren't there calzones?
Customer 5: I love the garlic pizza and can't get it anywhere else.
Customer 6: The garlic pizza is exceptional.
Customer 7: Pizza can get a little messy so I prefer a calzone's neatness.
Customer 8: Why is the pizza so expensive?
Customer 9: There's no way to do online ordering.
Customer 10: Why is the seating so uncomfortable and dirty?
"""

print("🤖 Computing design thinking interim results from customer comments ...")
pluginDT = kernel.import_semantic_skill_from_directory(skillsDirectory, "DesignThinking");
my_result = await kernel.run_async(pluginDT["Empathize"], pluginDT["Define"], input_str=customer_comments)
designthinking_str = "# Categorized observations\n"+str(my_result)
await kernel.memory.save_information_async(memoryCollectionName, id="designthinking", text=designthinking_str)

print("😶‍🌫️ 🎉 Memory is now flush with information :+)")


### 💁 In case you're wondering why we regenerated everything ...

That's because we weren't storing the information on disk or in the cloud and were using a vector database that writes to memory that was `volatile` — meaning that it is transient and doesn't stick around. To store information to a persistent vector database, in the Semantic Kernel universe there are many options like Azure Cognitive Search, Chroma, Weaviate, Qdrant, and a host of other awesome options. In this notebook we switched the Chroma option on, so the data will now live inside a folder `mymemories` and doesn't need to be regenerated. If you want to start over again, just delete the folder.

## 🎗️ Recall two of the business' strengths in the SWOT

In our early SWOT analysis, we distilled two possible differentiated strengths with this pizza store.

| STRENGTH 1 | STRENGTH 2 |
|---|---|
| The best garlic pizza in the business | The store owner was trained in Sicily |
| ![](assets/garlicpizza.png) | ![](assets/measuring.png) |

We take the tool and modify it a bit.

In [ ]:
from IPython.display import display, Markdown

topic = "The business makes the best garlic pizza and the owner was trained in Sicily."
swot_quality = "Strengths"
counter = 0

gathered_context = []
max_memories = 3
memories = await kernel.memory.search_async(memoryCollectionName, topic, limit=max_memories, min_relevance_score=0.75)

print(f"🤖 Leveraging information available to address '{topic}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

skillsDirectory = "./plugins-sk"
print(f"🤖 Synthesizing question list suitable for a research interview...")
pluginFC = kernel.import_semantic_skill_from_directory(skillsDirectory, "FriendlyConsultant");

my_context = kernel.create_new_context()
my_context['input'] = topic
my_context['context'] = "\n".join(gathered_context)
my_context['swot_quality'] = swot_quality

interview_result = await kernel.run_async(pluginFC["ResearchInterview"], input_context=my_context)

display(Markdown("# 🤖 Generated ...\n"+str(interview_result)))


## 📄 With the list of questions in hand, go ahead and get answers ...

In [ ]:
interview_transcript="""
Q: What specific training did the owner receive in Sicily and how has it been applied to the business?
A: The owner learned how to use a special kind of brick oven to make pizzas in Sicily.

Q: How has the unique garlic pizza recipe contributed to the success of the business?
A: Over half of the pizzas sold are the signature garlic pizza.

Q: What awards has the garlic pizza recipe won and how have these awards impacted the business?
A: The owner won the "best pizza of 2022" award from the local newspaper.

Q: How does the business currently utilize the owner's training and the unique garlic pizza recipe to its advantage?
A: The owner trains all new employees on how to make the garlic pizza.

Q: What feedback have customers given about the garlic pizza and the overall service of the business?
A: Customers come back for the garlic pizza.

Q: How does the business maintain the quality and consistency of the garlic pizza?
A: The owner has a simple recipe of flour, water, eggs, and garlic in specific and repeatable proportions.

Q: How does the business ensure that the skills and knowledge of the owner are passed on to the staff?
A: Right now it is all in the owner's head and shared verbally.

Q: What steps has the business taken to protect the unique garlic pizza recipe?
A: The recipe requires a special blend of cheeses that only the owner knows.

Q: How does the business plan to continue leveraging the owner's training and the unique garlic pizza recipe in the future?
A: The owner is looking for an apprentice to train.

Q: How does the business differentiate itself from competitors with similar offerings?
A: The garlic pizza is unique to this business.

Q: How does the business plan to address any potential weaknesses or threats that could impact the owner's training or the unique garlic pizza recipe?
A: Currently there's no plan to do so.
"""

skillsDirectory = "./plugins-sk"

topic = "How to protect the uniqueness of the garlic pizza recipe"
counter = 0
gathered_context = []
max_memories = 3
memories = await kernel.memory.search_async(memoryCollectionName, topic, limit=max_memories, min_relevance_score=0.75)
pluginFC = kernel.import_semantic_skill_from_directory(skillsDirectory, "FriendlyConsultant");

print(f"🤖 Leveraging information available to address '{topic}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

my_context = kernel.create_new_context()
my_context['input'] = topic
my_context['context'] = "\n".join(gathered_context) + "\n" + interview_transcript

preso_result = await kernel.run_async(pluginFC["Presentation"], input_context=my_context)

print(f"😶‍🌫️ New context for '{topic}' appended to vector database...")
await kernel.memory.save_information_async(memoryCollectionName, id=f"{topic}", text=topic + "\n\n" + str(preso_result))

display(Markdown("# 🤖 Generated ...\n"+str(preso_result)))


## 🍎 Training the trainers: Making it easy and painless

Recognizing the importance of the garlic pizza, it makes sense for the owner to get the recipe out of his head and into a training program. The owner does the work of writing down his expert tips for making the signature garlic pizza dish.

In [ ]:
what_is_this = "Recipe and process for making the signature garlic pizza"
expert_knowledge = """
Special Garlic Pizza Recipe:
> Ingredients: flour, eggs, sugar, garlic, salt, yeast, water, olive oil, tomato sauce, mozzarella cheese, parmesan cheese, basil

Special Cheese Blend Recipe:
> Ingredients: mozzarella cheese, parmesan cheese, cheddar cheese

Special Cooking Instructions:
> 1. Mix flour, eggs, sugar, garlic, salt, yeast, water, and olive oil in a bowl
> 2. Knead the dough
> 3. Let the dough rise
> 4. Roll out the dough
> 5. Spread tomato sauce on the dough
> 6. Sprinkle cheese blend and basil on the dough
> 7. Bake the pizza in the brick oven at 350 degrees for 10 minutes
> 8. Be sure to move the pizza around within the oven every 3 minutes
"""

print(f"😶‍🌫️ New context for '{what_is_this}' appended to vector database...")
await kernel.memory.save_information_async(memoryCollectionName, id=f"{what_is_this}", text=what_is_this + "\n\n" + expert_knowledge)

In [ ]:
skillsDirectory = "./plugins-sk"

topic = "How to make the signature garlic pizza and save time"
counter = 0
gathered_context = []
max_memories = 5
memories = await kernel.memory.search_async(memoryCollectionName, topic, limit=max_memories, min_relevance_score=0.75)
pluginFC = kernel.import_semantic_skill_from_directory(skillsDirectory, "FriendlyConsultant");

print(f"🤖 Leveraging information available to address '{topic}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

my_context = kernel.create_new_context()
my_context['input'] = topic
my_context['context'] = "\n".join(gathered_context)

training_result = await kernel.run_async(pluginFC["TrainingProgram"], input_context=my_context)

display(Markdown("# 🤖 Generated ...\n"+str(training_result)))